In [34]:
import pandas as pd
import numpy as np
import numpy.random as rn
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

In [39]:
def ITT_OLS(Y_N, Z_N, X_NM=None):
    """Intent-to-Treat (ITT) effect using ordinary least squares (OLS).
    
    Y_N:  Binary outcome.
    Z_N:  Binary instrument/assignment.
    X_NM: (Optional) Covariates.
    
    Returns:
        estimate, standard error 
    """
    Y_N = Y_N.astype(int)
    Z_N = Z_N.astype(int)
    ZX_NM = Z_N if X_NM is None else np.vstack((Z_N, X_NM.T)).T
    model = sm.OLS(Y_N, sm.add_constant(ZX_NM, prepend=False))
    results = model.fit()
    
    itt = results.params[0] * 100
    se = results.bse[0] * 100
    return itt, se

def CACE_2SLS(Y_N, Z_N, D_N, X_NM=None):
    """Compiler Average Causal Effect (CACE) effect
    estimated using 2-stage least squares (2SLS).
    
    Y_N:  Binary outcome.
    Z_N:  Binary instrument/assignment.
    D_N:  Binary treatment receipt.
    X_NM: (Optional) Covariates.
    
    Returns:
        estimate, standard error 
    """

    Y_N = Y_N.astype(int)
    Z_N = Z_N.astype(int)
    D_N = D_N.astype(int)
    
    DX_NM = D_N if X_NM is None else np.vstack((D_N, X_NM.T)).T
    ZX_NM = Z_N if X_NM is None else np.vstack((Z_N, X_NM.T)).T
    
    model = IV2SLS(endog=Y_N,
                   exog=sm.add_constant(DX_NM, prepend=False),
                   instrument=sm.add_constant(ZX_NM, prepend=False))

    results = model.fit()
    cace = results.params[0] * 100
    se = results.bse[0] * 100
    return cace, se

In [45]:
df_data = pd.read_csv('../dat/outvote_data.csv')

n = len(df_data)
print('n=%s eligible subjects' % f'{n:,}')

Z = df_data['Z'].to_numpy()  # assignment (1='assigned treatment', 0='assigned control')
D = df_data['D'].to_numpy()  # treatment receipt (1='messaged', 0='not messaged')
Y = df_data['Y'].to_numpy()  # voting outcome (1='voted', 0='did not vote')
M = df_data['M'].to_numpy()  # confident match to voter rolls (1='yes', 0='no')
Q = df_data['Q'].to_numpy()  # position in first queue (1,2,3... or NaN)

assign_rate = 100 * Z.mean()
print('%.2f%% of subjects assigned to receive treatment' % (assign_rate))

comply_rate = 100 * (D[Z==1].mean() - D[Z==0].mean())
print('%.2f%% compliance rate in eligible subject pool' % comply_rate)

good_match_rate = 100 * M.mean() 
print('%.2f%% of subjects are confidently matched' % good_match_rate)

nonnull_queue_rate = 100 * (1-np.isnan(Q).mean())
print('%.2f%% of subjects have non-null queue position' % nonnull_queue_rate)

itt, itt_se = ITT_OLS(Y, Z)
print('ITT=%.2f (SE=%.2f) percentage points' % (itt, itt_se))

cace, cace_se = CACE_2SLS(Y, Z, D)
print('CACE=%.2f (SE=%.2f) percentage points' % (cace, cace_se))

n=195,118 eligible subjects
94.91% of subjects assigned to receive treatment
16.41% compliance rate in eligible subject pool
28.78% of subjects are confidently matched
59.63% of subjects have non-null queue position
ITT=0.40 (SE=0.47) percentage points
CACE=2.42 (SE=2.84) percentage points


In [46]:
df_study = df_data.loc[(df_data['M'] == 1) &   # only confidently matched
                       (df_data['Q'] <= 103)]  # queue position <= q_max=103

Z = df_study['Z'].to_numpy()  # assignment (1='assigned treatment', 0='assigned control')
D = df_study['D'].to_numpy()  # treatment receipt (1='messaged', 0='not messaged')
Y = df_study['Y'].to_numpy()  # voting outcome (1='voted', 0='did not vote')

itt, itt_se = ITT_OLS(Y, Z)
print('ITT=%.2f (SE=%.2f) percentage points' % (itt, itt_se))

cace, cace_se = CACE_2SLS(Y, Z, D)
print('CACE=%.2f (SE=%.2f) percentage points' % (cace, cace_se))

ITT=3.02 (SE=1.10) percentage points
CACE=11.88 (SE=4.37) percentage points


In [28]:
queue_nan_rate

array([  0,   0,   0, ..., 100, 100,   0])